In [ ]:
import pandas as pd
import random
from datetime import timedelta
import vectorbtpro as vbt

from vctr.trading.utils.misc import tradable_coins
from models.lstm.main import load_model, save_model, predict

# save_model(model, 'lstm-prod-mk-8')
model = load_model('latest')


def backtest_segment(df):
    pf = vbt.Portfolio.from_signals(
        df['close'],
        df['pred'] == 1,
        df['pred'] == 2,
        freq='1h',
        init_cash=10000,
        tsl_stop=0.005,
        sl_stop=0.05,
        fees=0.0006,
        slippage=0.001,
    )
    return pf.stats('total_return').values[0]


def the_wringer(symbol):
    # Step 1: Get the data for the symbol
    df = predict(model, symbol, '1h', start='2017-01-01')

    # Step 2: Initialize an empty dataframe to store the results
    results_df = pd.DataFrame(columns=['symbol', 'start_date', 'end_date', 'return_pct'])

    # Step 3: Define the number of permutations and the maximum segment length
    num_permutations = 1000
    max_segment_length = 365  # in days

    # Step 4: Loop through each permutation
    for _ in range(num_permutations):
        # Step 5: Choose a random start date
        start_date = df.index[random.randint(0, len(df) - max_segment_length)]

        # Step 6: Choose a random end date between start_date and the minimum of the current maximum date in the dataset and start_date plus max_segment_length days
        max_end_date = min(start_date + timedelta(days=max_segment_length), df.index.max())
        end_date = start_date + timedelta(days=random.randint(1, (max_end_date - start_date).days))

        # Step 7: Slice the dataframe using the start and end dates
        slice_df = df.loc[start_date:end_date]

        # Step 8: Call the backtest_segment function and store the result
        return_pct = backtest_segment(slice_df)
        result = {
            'symbol': symbol,
            'start_date': start_date,
            'end_date': end_date,
            'duration': end_date - start_date,
            'return_pct': return_pct,
        }

        results_df = pd.concat([results_df, pd.DataFrame(result, index=[0])], ignore_index=True)

    # Step 9: Return the results dataframe
    return results_df


In [ ]:
import random
from models.lstm.main import load_model

# model = load_model('lstm-prod-mk-8')

res = the_wringer(random.choice(tradable_coins))
res.sort_values('return_pct', ascending=True).head(10)
res[res['return_pct'] < 0].count()


In [ ]:
res.sort_values('return_pct', ascending=True).head(22)